## Dash

In [1]:
import dash
from dash import html
from dash import dcc
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output

In [2]:
df = pd.read_csv('turma_matricula_docente_filtrados.csv')

In [3]:
app = dash.Dash(__name__)

In [4]:
app.layout = html.Div([
    html.H1("Taxa de Aprovação por Curso, Componente e Docente"),
    
    dcc.Dropdown(
        id='unidade-dropdown',
        options=[{'label': i, 'value': i} for i in df['unidade_responsavel'].unique()],
        value=df['unidade_responsavel'].unique()[0],
        multi=False
    ),
    
    dcc.Dropdown(
        id='componente-dropdown',
        multi=False
    ),
    
    dcc.Dropdown(
        id='docente-dropdown',
        multi=False
    ),
    
    dcc.Graph(
        id='pie-chart'
    )
])

In [5]:
@app.callback(
    Output('componente-dropdown', 'options'),
    Output('componente-dropdown', 'value'),
    Input('unidade-dropdown', 'value')
)
def update_componente_dropdown(unidade):
    df_unidade = df[df['unidade_responsavel'] == unidade]
    options = [{'label': i, 'value': i} for i in df_unidade['nome_componente'].unique()]
    value = df_unidade['nome_componente'].unique()[0] if df_unidade['nome_componente'].unique().size > 0 else None
    return options, value

In [6]:
@app.callback(
    Output('docente-dropdown', 'options'),
    Output('docente-dropdown', 'value'),
    Input('unidade-dropdown', 'value'),
    Input('componente-dropdown', 'value')
)
def update_docente_dropdown(unidade, componente):
    df_componente = df[(df['unidade_responsavel'] == unidade) & (df['nome_componente'] == componente)]
    options = [{'label': i, 'value': i} for i in df_componente['nome_docente'].unique()]
    value = df_componente['nome_docente'].unique()[0] if df_componente['nome_docente'].unique().size > 0 else None
    return options, value

In [7]:
@app.callback(
    Output('pie-chart', 'figure'),
    Input('unidade-dropdown', 'value'),
    Input('componente-dropdown', 'value'),
    Input('docente-dropdown', 'value')
)
def update_graph(unidade, componente, docente):
    df_filtrado = df[
        (df['unidade_responsavel'] == unidade) &
        (df['nome_componente'] == componente) &
        (df['nome_docente'] == docente)
    ].drop_duplicates(subset='discente')

    contagem_descricao = df_filtrado['descricao'].value_counts()
    fig = px.pie(df_filtrado, names=contagem_descricao.index, values=contagem_descricao.values, title='Taxa de Aprovação')

    return fig

In [8]:
if __name__ == '__main__':
    app.run_server(debug=True)